This example notebook shows how we can train an [image/digit classification](https://pytorch.org/tutorials/beginner/nn_tutorial.html?highlight=mnist)
model based on MNIST dataset, and store it as TileDB array. Firstly, let's import what we need.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import tiledb
import glob
import json
import pickle
import os

from pprint import pprint


from tiledb.ml.models.pytorch import PyTorchTileDB

First let's define the parameters/hyperparameters we will need.

In [2]:
epochs = 1
batch_size_train = 128
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

# Set random seeds for anything using random number generation
random_seed = 1

# Disable nondeterministic algorithms
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

We  will also need the DataLoaders API for the dataset. We will also employ TorchVision which let's as load the MNIST
dataset in a handy way. We'll use a batch_size of 64 for training while the values 0.1307 and 0.3081 used for
the Normalize() transformation below are the global mean and standard deviation of the MNIST dataset,
we'll take them as a given here.

In [3]:
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)


Moving on, we build our network. We'll use two 2-D convolutional layers followed by two fully-connected
layers. As activation function we'll choose ReLUs and as a means of regularization we'll use two dropout layers.

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)


We will now initialise our Neural Network and optimizer.

In [5]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

We continue with the training loop and we iterate over all training data once per epoch. Loading the individual batches
is handled by the DataLoader. We need to set the gradients to zero using optimizer.zero_grad() since PyTorch by default
accumulates gradients. We then produce the output of the network (forward pass) and compute a negative log-likelihodd
loss between the output and the ground truth label. The backward() call we now collect a new set of gradients which we
propagate back into each of the network's parameters using optimizer.step().

In [6]:
train_losses = []
train_counter = []

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))

for epoch in range(1, epochs + 1):
  train(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316989
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.310947
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.287661
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.261894
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.287373
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.252142
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.202135
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.180613
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.127905
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.048942
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.947829
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.886966
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.879172
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.631594
Train Epoch: 1 [17920/60000 (30%)]	Loss: 1.653769
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.471534
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.364005
Train Epoch: 1 [21760/60000 (36%)]	Loss: 1.362187
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.254044
Train Epoch: 1 [24320/60000 (41%)]	Loss: 1.181683
Train Epoch: 1 [

We can now save the trained model as a TileDB array. In case we want to train  the model further in a later time, we can save
optimizer's state_dict in our TileDB array. In case we will use our model only for inference, we don't have to save optimizer's
state_dict and we only keep model's state_dict. We first declare a PytTorchTileDB object (with the corresponding uri) and then
save the model as a TileDB array. Finally, we can save any kind of meta data (in any structure, i.e., list, tuple or dictionary)
by passing a dictionary to the meta attribute.

In [7]:
tiledb_model_1 = PyTorchTileDB(uri='tiledb-pytorch-mnist-1')

tiledb_model_1.save(update=False, model_info={
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    },
                    meta={'epochs': epochs,
                          'train_loss': train_losses})

The above step will create a TileDB array in your working directory. For information about the structure of a dense
TileDB array in terms of files on disk please take a look [here](https://docs.tiledb.com/main/basic-concepts/data-format).
Let's open our TileDB array model and check metadata. Metadata that are of type list, dict or tuple have been JSON
serialized while saving, i.e., we need json.loads to deserialize them.

In [8]:
# Check array directory
pprint(glob.glob('tiledb-pytorch-mnist-1/*'))

# Open in write mode in order to add metadata
print()
model_array_1 = tiledb.open('tiledb-pytorch-mnist-1')
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes):
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value))

['tiledb-pytorch-mnist-1/__meta',
 'tiledb-pytorch-mnist-1/__lock.tdb',
 'tiledb-pytorch-mnist-1/__array_schema.tdb',
 'tiledb-pytorch-mnist-1/__1617026164071_1617026164071_66e47eea64e44eb9bc296acb9955988a_8',
 'tiledb-pytorch-mnist-1/__1617026164071_1617026164071_66e47eea64e44eb9bc296acb9955988a_8.ok']


Key: epochs, Value: 1
Key: python_version, Value: 3.7.3
Key: pytorch_version, Value: 1.8.1
Key: train_loss, Value: [2.3169891834259033, 2.3109469413757324, 2.287660837173462, 2.2618942260742188, 2.2873728275299072, 2.2521421909332275, 2.202134609222412, 2.180612802505493, 2.1279051303863525, 2.0489416122436523, 1.9478294849395752, 1.8869657516479492, 1.8791720867156982, 1.6315940618515015, 1.65376877784729, 1.4715337753295898, 1.3640053272247314, 1.3621865510940552, 1.2540440559387207, 1.1816833019256592, 1.1333627700805664, 1.1177345514297485, 1.0250669717788696, 1.0229164361953735, 0.8689796328544617, 0.9791150093078613, 0.9290736317634583, 0.9272529482841492, 0.8364390134811401, 0.

As we can see, in array's metadata we have by default information about the backend we used for training (pytorch),
pytorch version, python version and the extra metadata about epochs and training loss that we added.
We can load and check any of the aforementioned without having to load the entire model in memory.
Moreover, we can add any kind of extra information in model's metadata also by opening the TileDB array and adding new keys.

In [9]:
# Open the array in write mode
with tiledb.Array('tiledb-pytorch-mnist-1', "w") as A:
    # Keep all history
    A.meta['new_meta'] = json.dumps(['Any kind of info'])

# Check that everything is there
model_array_1 = tiledb.open('tiledb-pytorch-mnist-1')
for key, value in model_array_1.meta.items():
    if isinstance(value, bytes):
        value = json.loads(value)
    print("Key: {}, Value: {}".format(key, value))

Key: epochs, Value: 1
Key: new_meta, Value: ["Any kind of info"]
Key: python_version, Value: 3.7.3
Key: pytorch_version, Value: 1.8.1
Key: train_loss, Value: [2.3169891834259033, 2.3109469413757324, 2.287660837173462, 2.2618942260742188, 2.2873728275299072, 2.2521421909332275, 2.202134609222412, 2.180612802505493, 2.1279051303863525, 2.0489416122436523, 1.9478294849395752, 1.8869657516479492, 1.8791720867156982, 1.6315940618515015, 1.65376877784729, 1.4715337753295898, 1.3640053272247314, 1.3621865510940552, 1.2540440559387207, 1.1816833019256592, 1.1333627700805664, 1.1177345514297485, 1.0250669717788696, 1.0229164361953735, 0.8689796328544617, 0.9791150093078613, 0.9290736317634583, 0.9272529482841492, 0.8364390134811401, 0.9574681520462036, 0.9235935211181641, 0.7311012148857117, 0.7084848880767822, 0.868228554725647, 0.6333558559417725, 0.6871156096458435, 0.7291515469551086, 0.7824904918670654, 0.7346636652946472, 0.6073712110519409, 0.74458247423172, 0.708631157875061, 0.62883228

For the case of PyTorch models, as mentioned above, we save model's state_dict and optimizer's state_dict,
as [variable sized attributes)](https://docs.tiledb.com/main/solutions/tiledb-embedded/api-usage/writing-arrays/var-length-attributes)
(pickled), i.e., we can open the TileDB and get only the state_dict of the model or optimizer,
without bringing the whole model in memory. For example, we can load model's and optimizer's state_dict
for model tiledb-pytorch-mnist-1 as follows.

In [11]:
# First open arrays
model_array_1 = tiledb.open('tiledb-pytorch-mnist-1')[:]

# Load model state_dict
model_1_state_dict = pickle.loads(model_array_1['model_state_dict'].item(0))

# Load optimizer state_dict
optimizer_1_state_dict = pickle.loads(model_array_1['optimizer_state_dict'].item(0))

print(model_1_state_dict)
print(optimizer_1_state_dict)

OrderedDict([('conv1.weight', tensor([[[[ 0.1196, -0.0909, -0.0716,  0.0518, -0.2079],
          [ 0.1540, -0.0288,  0.0913,  0.0247, -0.0133],
          [ 0.1030,  0.0503,  0.1028, -0.0420,  0.0180],
          [ 0.0134,  0.0608,  0.0284,  0.2031,  0.0921],
          [-0.0730, -0.1239, -0.0435, -0.0908, -0.0517]]],


        [[[ 0.0473,  0.1834,  0.1117, -0.2845,  0.0098],
          [ 0.1284,  0.2763,  0.1337, -0.2607, -0.2752],
          [ 0.0143,  0.2913, -0.0139,  0.0315, -0.1523],
          [ 0.3000,  0.0248,  0.1901, -0.0017, -0.1102],
          [ 0.1880, -0.0187,  0.1166, -0.0169,  0.0175]]],


        [[[-0.2377, -0.1176,  0.1562,  0.0474,  0.2775],
          [ 0.1391,  0.0210, -0.0236,  0.2540,  0.1335],
          [-0.1081,  0.2079,  0.2517,  0.2953, -0.0805],
          [ 0.0130,  0.0836,  0.1333, -0.0921, -0.1461],
          [ 0.1425,  0.0148,  0.1544,  0.0294,  0.0532]]],


        [[[ 0.0704,  0.1073,  0.1291, -0.1668,  0.1078],
          [-0.1428, -0.1451,  0.0154, -0.0328,

Moving on, we can load the trained models for prediction, evaluation or retraining, as usual with
PyTorch models.

In [12]:
# Place holder for the loaded model
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

# Load returns possible extra attributes, other than model's and optimizer's state dicts. In case there were
# no extra attributes it will return an empty dict
_ = tiledb_model_1.load(model=network, optimizer=optimizer)

What is really nice with saving models as TileDB array, is native versioning based on fragments as described
[here](https://docs.tiledb.com/main/basic-concepts/data-format#immutable-fragments). We can load a model, retrain it
with new data and update the already existing TileDB model array with new model parameters and metadata. All information, old
and new will be there and accessible. This is extremely useful when you retrain with new data or trying different architectures for the same
problem, and you want to keep track of all your experiments without having to store different model instances. In our case,
let's continue training model_1 with the rest of our dataset and for 2 more epochs. After training is done, you will
notice the extra directories and files (fragments) added to tiledb-keras-mnist-sequential-1 TileDB array directory,
which keep all versions of the model.

In [13]:
train_losses = []
train_counter = []

# We train for some extra 2 epochs
for epoch in range(1, 2 + 1):
  train(epoch)

# and update
tiledb_model_1.save(update=True, model_info={
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    },
                    meta={'epochs': epochs,
                          'train_loss': train_losses})

# Check array directory
print()
pprint(glob.glob('tiledb-pytorch-mnist-1/*'))

# tiledb.array_fragments() requires TileDB-Py version > 0.8.5
fragments_info = tiledb.array_fragments('tiledb-pytorch-mnist-1')

print()
print("====== FRAGMENTS  INFO ======")
print("array uri: {}".format(fragments_info.array_uri))
print("number of fragments: {}".format(len(fragments_info)))

for fragment_num, fragment in enumerate(fragments_info, start=1):
    print()
    print("===== FRAGMENT NUMBER {} =====".format(fragment.num))
    print("fragment uri: {}".format(fragment.uri))
    print("is dense: {}".format(fragment.dense))
    print("is sparse: {}".format(fragment.sparse))
    print("cell num: {}".format(fragment.cell_num))
    print("has consolidated metadata: {}".format(fragment.has_consolidated_metadata))
    print("non empty domain: {}".format(fragment.non_empty_domain))
    print("timestamp range: {}".format(fragment.timestamp_range))
    print("number of fragments to vacuum: {}".format(fragment.to_vacuum_num))
    print("uri of fragments to vacuum: {}".format(fragment.to_vacuum_uri))
    print(
        "number of unconsolidated metadata: {}".format(
            fragment.unconsolidated_metadata_num
        )
    )
    print("version: {}".format(fragment.version))

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.573581
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.668981
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.641363
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.613981
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.729578
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.647342
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.523893
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.522834
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.553014
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.631665
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.675059
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.577137
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.690339
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.560791
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.448240
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.499283
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.514028
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.574534
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.563722
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.576612
Train Epoch: 1 [

Finally, a very interesting and useful, for machine learning models, TileDB feature that is described
[here](https://docs.tiledb.com/main/basic-concepts/data-format#groups) and [here](https://docs.tiledb.com/main/solutions/tiledb-embedded/api-usage/object-management#creating-tiledb-groups)
are groups. Assuming we want to solve the MNIST problem, and we want to try several architectures. We can save each architecture
as a separate TileDB array with native versioning each time it is re-trained, and then organise all models that solve the same problem (MNIST)
as a TileDB array group with any kind of hierarchy. Let's firstly define a new model architecture.

In [14]:
class OtherNet(nn.Module):
    # For the sake of simplicity we just tweak the initial architecture by replacing a relu with relu6.
    def __init__(self):
        super(OtherNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu6(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)

Then train it and save it as a new TileDB array.

In [ ]:
network = OtherNet()
optimizer = optim.Adam(network.parameters(), lr=learning_rate)

train_losses = []
train_counter = []

for epoch in range(1, epochs + 1):
    train(epoch)

tiledb_model_2 = PyTorchTileDB(uri='tiledb-pytorch-mnist-2')

tiledb_model_2.save(update=False, model_info={
    'model_state_dict': network.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    },
                    meta={'epochs': epochs,
                          'train_loss': train_losses})

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.337260
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.897245
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.235484
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.009100
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.129466
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.807468
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.628168
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.623165
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.525108
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.639337
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.460995
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.564493
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.600411
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.707955
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.619303
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.562256
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.577714
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.474819
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.435444
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.694258
Train Epoch: 1 [

Now we can create a TileDB group and organise (in hierarchies, e.g., sophisticated vs less sophisticated) all our
MNIST models as follows.

In [ ]:
tiledb.group_create('MNIST_Group')
os.system('mv tiledb-pytorch-mnist-1 MNIST_Group/')
os.system('mv tiledb-pytorch-mnist-2 MNIST_Group/')

Any time we can check and query all the available models, including their metadata, for a specific problem like MNIST.

In [18]:
tiledb.ls('MNIST_Group', lambda obj_path, obj_type: print(obj_path, obj_type))

file:///Users/george/PycharmProjects/TileDB-ML/example_notebooks/models/MNIST_Group/tiledb-pytorch-mnist-1 array
file:///Users/george/PycharmProjects/TileDB-ML/example_notebooks/models/MNIST_Group/tiledb-pytorch-mnist-2 array
